In [35]:
import os
import re
from bs4 import BeautifulSoup
import json
from pathlib import Path
import shutil
from datetime import datetime

class HTMLMetaChecker:
    def __init__(self):
        self.results = []
        self.expected_domain = "https://emfls.github.io/kor/report/travel/"
        self.expected_author = "Itagi"
        self.adsense_code = 'ca-pub-8830524482034754'
        self.analytics_code = 'G-QP5Q67GE5B'
        
    def extract_filename_from_path(self, file_path):
        """파일 경로에서 파일명 추출 (확장자 제외)"""
        return Path(file_path).stem
    
    def check_domain_match(self, url, filename):
        """URL이 예상 도메인과 파일명 패턴과 일치하는지 확인"""
        if not url:
            return False, "URL이 없습니다"
        
        expected_url = f"{self.expected_domain}{filename}.html"
        if url == expected_url:
            return True, "일치"
        else:
            return False, f"예상: {expected_url}, 실제: {url}"
    
    def fix_html_file(self, file_path, issues):
        """HTML 파일의 문제점들을 자동으로 수정"""
        try:
            # 백업 생성
            backup_path = f"{file_path}.backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            shutil.copy2(file_path, backup_path)
            print(f"   백업 생성: {backup_path}")
            
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            
            soup = BeautifulSoup(content, 'html.parser')
            filename = self.extract_filename_from_path(file_path)
            expected_url = f"{self.expected_domain}{filename}.html"
            
            fixed_issues = []
            
            # 1. og:url 수정
            if any('og:url' in issue for issue in issues):
                og_url_tag = soup.find('meta', property='og:url')
                if og_url_tag:
                    og_url_tag['content'] = expected_url
                    fixed_issues.append('og:url 수정됨')
                else:
                    # og:url 태그가 없으면 생성
                    new_og_url = soup.new_tag('meta', property='og:url', content=expected_url)
                    head = soup.find('head')
                    if head:
                        head.append(new_og_url)
                        fixed_issues.append('og:url 태그 추가됨')
            
            # 2. canonical URL 수정
            if any('canonical' in issue for issue in issues):
                canonical_tag = soup.find('link', rel='canonical')
                if canonical_tag:
                    canonical_tag['href'] = expected_url
                    fixed_issues.append('canonical URL 수정됨')
                else:
                    # canonical 태그가 없으면 생성
                    new_canonical = soup.new_tag('link', rel='canonical', href=expected_url)
                    head = soup.find('head')
                    if head:
                        head.append(new_canonical)
                        fixed_issues.append('canonical 태그 추가됨')
            
            # 3. Structured Data URL 수정
            if any('Structured Data' in issue for issue in issues):
                structured_data_scripts = soup.find_all('script', type='application/ld+json')
                for script in structured_data_scripts:
                    try:
                        data = json.loads(script.string)
                        if 'url' in data:
                            data['url'] = expected_url
                            script.string = json.dumps(data, ensure_ascii=False, indent=4)
                            fixed_issues.append('Structured Data URL 수정됨')
                            break
                    except json.JSONDecodeError:
                        continue
                else:
                    # Structured Data가 없으면 생성
                    structured_data = {
                        "@context": "https://schema.org",
                        "@type": "TravelGuide",
                        "name": soup.find('title').get_text() if soup.find('title') else f"{filename} 여행 가이드",
                        "description": soup.find('meta', attrs={'name': 'description'})['content'] if soup.find('meta', attrs={'name': 'description'}) else "",
                        "url": expected_url
                    }
                    new_script = soup.new_tag('script', type='application/ld+json')
                    new_script.string = json.dumps(structured_data, ensure_ascii=False, indent=4)
                    head = soup.find('head')
                    if head:
                        head.append(new_script)
                        fixed_issues.append('Structured Data 추가됨')
            
            # 4. meta author 수정
            if any('author' in issue for issue in issues):
                meta_author_tag = soup.find('meta', attrs={'name': 'author'})
                if meta_author_tag:
                    meta_author_tag['content'] = self.expected_author
                    fixed_issues.append('meta author 수정됨')
                else:
                    # meta author 태그가 없으면 생성
                    new_author = soup.new_tag('meta', name='author', content=self.expected_author)
                    head = soup.find('head')
                    if head:
                        head.append(new_author)
                        fixed_issues.append('meta author 태그 추가됨')
            
            # 5. 애드센스 코드 추가
            if any('애드센스' in issue for issue in issues):
                if not re.search(r'googlesyndication\.com|ca-pub-\d+', content):
                    adsense_script = soup.new_tag('script', attrs={'async': True}, 
                    src=f"https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client={self.adsense_code}",
                    crossorigin="anonymous")
                    head = soup.find('head')
                    if head:
                        head.append(adsense_script)
                        fixed_issues.append('애드센스 코드 추가됨')
            
            # 6. 구글 애널리틱스 코드 추가
            if any('애널리틱스' in issue for issue in issues):
                if not re.search(r'googletagmanager\.com/gtag|G-[A-Z0-9]+', content):
                    # gtag 스크립트 추가
                    gtag_script1 = soup.new_tag('script', attrs={'async': True},
                    src=f"https://www.googletagmanager.com/gtag/js?id={self.analytics_code}")
                    gtag_script2 = soup.new_tag('script')
                    gtag_script2.string = f"""
                    window.dataLayer = window.dataLayer || [];
                    function gtag(){{dataLayer.push(arguments);}}
                    gtag('js', new Date());
                    gtag('config', '{self.analytics_code}');
                    """
                    head = soup.find('head')
                    if head:
                        head.append(gtag_script1)
                        head.append(gtag_script2)
                        fixed_issues.append('구글 애널리틱스 코드 추가됨')
            
            # 수정된 HTML을 파일에 저장
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(str(soup))
            
            return fixed_issues
            
        except Exception as e:
            return [f'수정 중 오류 발생: {str(e)}']
    
    def check_html_file(self, file_path):
        """개별 HTML 파일 검사"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            
            soup = BeautifulSoup(content, 'html.parser')
            filename = self.extract_filename_from_path(file_path)
            
            # 검사 결과 저장용 딕셔너리
            result = {
                'file_path': file_path,
                'filename': filename,
                'og_url': None,
                'canonical_url': None,
                'structured_data_url': None,
                'meta_author': None,
                'has_adsense': False,
                'has_analytics': False,
                'urls_match_filename': False,
                'author_is_itagi': False,
                'domain_matches': False,
                'issues': [],
                'fixed_issues': []
            }
            
            # 1. og:url 메타태그 검사
            og_url_tag = soup.find('meta', property='og:url')
            if og_url_tag:
                result['og_url'] = og_url_tag.get('content')
                domain_match, domain_msg = self.check_domain_match(result['og_url'], filename)
                if not domain_match:
                    result['issues'].append(f'og:url 도메인 불일치: {domain_msg}')
            else:
                result['issues'].append('og:url 메타태그가 없습니다')
            
            # 2. canonical URL 검사
            canonical_tag = soup.find('link', rel='canonical')
            if canonical_tag:
                result['canonical_url'] = canonical_tag.get('href')
                domain_match, domain_msg = self.check_domain_match(result['canonical_url'], filename)
                if not domain_match:
                    result['issues'].append(f'canonical URL 도메인 불일치: {domain_msg}')
            else:
                result['issues'].append('canonical URL이 없습니다')
            
            # 3. Structured Data에서 URL 검사
            structured_data_scripts = soup.find_all('script', type='application/ld+json')
            for script in structured_data_scripts:
                try:
                    data = json.loads(script.string)
                    if 'url' in data:
                        result['structured_data_url'] = data['url']
                        domain_match, domain_msg = self.check_domain_match(result['structured_data_url'], filename)
                        if not domain_match:
                            result['issues'].append(f'Structured Data URL 도메인 불일치: {domain_msg}')
                        break
                except json.JSONDecodeError:
                    continue
            
            if not result['structured_data_url']:
                result['issues'].append('Structured Data에 URL이 없습니다')
            
            # 4. author 메타태그 검사
            meta_author_tag = soup.find('meta', attrs={'name': 'author'})
            if meta_author_tag:
                result['meta_author'] = meta_author_tag.get('content')
                if result['meta_author'] == self.expected_author:
                    result['author_is_itagi'] = True
                else:
                    result['issues'].append(f"meta author이 '{self.expected_author}'가 아닙니다: {result['meta_author']}")
            else:
                result['issues'].append('meta author 태그가 없습니다')
            
            # 5. 애드센스 코드 검사
            adsense_pattern = r'googlesyndication\.com|ca-pub-\d+'
            if re.search(adsense_pattern, content):
                result['has_adsense'] = True
            else:
                result['issues'].append('애드센스 코드가 없습니다')
            
            # 6. 구글 애널리틱스 코드 검사
            analytics_pattern = r'googletagmanager\.com/gtag|G-[A-Z0-9]+'
            if re.search(analytics_pattern, content):
                result['has_analytics'] = True
            else:
                result['issues'].append('구글 애널리틱스 코드가 없습니다')
            
            # 7. 전체 URL 일치성 검사
            urls_to_check = [result['og_url'], result['canonical_url'], result['structured_data_url']]
            expected_url = f"{self.expected_domain}{filename}.html"
            
            all_urls_match = True
            for url in urls_to_check:
                if url and url != expected_url:
                    all_urls_match = False
                    break
            
            result['urls_match_filename'] = all_urls_match
            result['domain_matches'] = all_urls_match
            
            return result
            
        except Exception as e:
            return {
                'file_path': file_path,
                'filename': filename,
                'error': str(e),
                'issues': [f'파일 처리 중 오류 발생: {str(e)}']
            }
    
    def check_directory(self, directory_path):
        """디렉토리 내 모든 HTML 파일 검사"""
        html_files = []
        
        # HTML 파일 찾기
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                if file.endswith('.html'):
                    html_files.append(os.path.join(root, file))
        
        print(f"총 {len(html_files)}개의 HTML 파일을 발견했습니다.")
        
        # 각 파일 검사
        for i, file_path in enumerate(html_files, 1):
            print(f"검사 중... ({i}/{len(html_files)}) {file_path}")
            result = self.check_html_file(file_path)
            self.results.append(result)
        
        return self.results
    
    def auto_fix_problems(self):
        """문제가 있는 파일들을 자동으로 수정"""
        problem_files = [r for r in self.results if r.get('issues')]
        
        if not problem_files:
            print("수정할 문제가 없습니다.")
            return
        
        print(f"\n🔧 {len(problem_files)}개 파일의 문제를 자동 수정합니다...")
        
        for i, result in enumerate(problem_files, 1):
            print(f"\n수정 중... ({i}/{len(problem_files)}) {result['file_path']}")
            
            if 'error' in result:
                print(f"   ❌ 오류로 인해 수정할 수 없습니다: {result['error']}")
                continue
            
            fixed_issues = self.fix_html_file(result['file_path'], result['issues'])
            result['fixed_issues'] = fixed_issues
            
            if fixed_issues:
                print(f"   ✅ 수정 완료: {', '.join(fixed_issues)}")
            else:
                print(f"   ⚠️ 수정할 수 없었습니다")
    
    def generate_report(self, output_file='html_check_report.txt', problems_only=True):
        """검사 결과 리포트 생성 (문제가 있는 파일만 표시 옵션)"""
        # 문제가 있는 파일만 필터링
        if problems_only:
            problem_results = [r for r in self.results if r.get('issues')]
        else:
            problem_results = self.results
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("HTML 메타태그 및 코드 검사 리포트\n")
            f.write("=" * 80 + "\n\n")
            
            # 전체 통계
            total_files = len(self.results)
            files_with_issues = len([r for r in self.results if r.get('issues')])
            files_fixed = len([r for r in self.results if r.get('fixed_issues')])
            files_without_adsense = len([r for r in self.results if not r.get('has_adsense', False)])
            files_without_analytics = len([r for r in self.results if not r.get('has_analytics', False)])
            files_with_domain_mismatch = len([r for r in self.results if not r.get('domain_matches', False)])
            files_without_itagi_author = len([r for r in self.results if not r.get('author_is_itagi', False)])
            
            f.write(f"📊 전체 통계\n")
            f.write(f"- 총 검사 파일 수: {total_files}\n")
            f.write(f"- 문제가 있는 파일 수: {files_with_issues}\n")
            f.write(f"- 자동 수정된 파일 수: {files_fixed}\n")
            f.write(f"- 도메인 불일치 파일: {files_with_domain_mismatch}\n")
            f.write(f"- 애드센스 코드 없는 파일: {files_without_adsense}\n")
            f.write(f"- 애널리틱스 코드 없는 파일: {files_without_analytics}\n")
            f.write(f"- author가 '{self.expected_author}'가 아닌 파일: {files_without_itagi_author}\n")
            f.write(f"- 예상 도메인: {self.expected_domain}\n\n")
            
            if problems_only:
                f.write("⚠️ 문제가 있는 파일만 표시\n")
                f.write("-" * 80 + "\n")
            else:
                f.write("📋 모든 파일 검사 결과\n")
                f.write("-" * 80 + "\n")
            
            if not problem_results:
                f.write("\n🎉 모든 파일이 정상입니다!\n")
                return
            
            for result in problem_results:
                f.write(f"\n🔍 파일: {result['file_path']}\n")
                f.write(f"   파일명: {result.get('filename', 'N/A')}\n")
                
                if 'error' in result:
                    f.write(f"   ❌ 오류: {result['error']}\n")
                    continue
                
                f.write(f"   og:url: {result.get('og_url', 'N/A')}\n")
                f.write(f"   canonical: {result.get('canonical_url', 'N/A')}\n")
                f.write(f"   structured_data_url: {result.get('structured_data_url', 'N/A')}\n")
                f.write(f"   meta_author: {result.get('meta_author', 'N/A')}\n")
                f.write(f"   애드센스: {'✅' if result.get('has_adsense') else '❌'}\n")
                f.write(f"   애널리틱스: {'✅' if result.get('has_analytics') else '❌'}\n")
                f.write(f"   도메인 일치: {'✅' if result.get('domain_matches') else '❌'}\n")
                f.write(f"   author='{self.expected_author}': {'✅' if result.get('author_is_itagi') else '❌'}\n")
                
                if result.get('issues'):
                    f.write(f"   ⚠️ 발견된 문제점:\n")
                    for issue in result['issues']:
                        f.write(f"      - {issue}\n")
                
                if result.get('fixed_issues'):
                    f.write(f"   🔧 자동 수정된 항목:\n")
                    for fixed in result['fixed_issues']:
                        f.write(f"      - {fixed}\n")
        
        print(f"\n리포트가 '{output_file}' 파일로 저장되었습니다.")
    
    def generate_csv_report(self, output_file='html_check_report.csv', problems_only=True):
        """CSV 형태의 리포트 생성 (문제가 있는 파일만 표시 옵션)"""
        import csv
        
        # 문제가 있는 파일만 필터링
        if problems_only:
            results_to_export = [r for r in self.results if r.get('issues')]
        else:
            results_to_export = self.results
        
        with open(output_file, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            
            # 헤더 작성
            writer.writerow([
                '파일경로', '파일명', 'og:url', 'canonical URL', 'structured_data_url', 'meta_author',
                '애드센스', '애널리틱스', '도메인일치', f'author={self.expected_author}', '문제점', '수정된항목'
            ])
            
            # 데이터 작성
            for result in results_to_export:
                writer.writerow([
                    result['file_path'],
                    result.get('filename', ''),
                    result.get('og_url', ''),
                    result.get('canonical_url', ''),
                    result.get('structured_data_url', ''),
                    result.get('meta_author', ''),
                    'O' if result.get('has_adsense') else 'X',
                    'O' if result.get('has_analytics') else 'X',
                    'O' if result.get('domain_matches') else 'X',
                    'O' if result.get('author_is_itagi') else 'X',
                    ' | '.join(result.get('issues', [])),
                    ' | '.join(result.get('fixed_issues', []))
                ])
        
        print(f"CSV 리포트가 '{output_file}' 파일로 저장되었습니다.")

def main():
    """메인 실행 함수"""
    checker = HTMLMetaChecker()
    
    # 검사할 디렉토리 경로 입력
    directory_path = input("검사할 디렉토리 경로를 입력하세요 (현재 디렉토리는 '.'): ").strip()
    if not directory_path:
        directory_path = '.'
    
    if not os.path.exists(directory_path):
        print("❌ 지정된 경로가 존재하지 않습니다.")
        return
    
    # 도메인 설정 확인
    print(f"📂 '{directory_path}' 디렉토리를 검사합니다...")
    print(f"🌐 예상 도메인: {checker.expected_domain}")
    print(f"👤 예상 작성자: {checker.expected_author}")
    
    # 설정 변경 옵션
    change_settings = input("설정을 변경하시겠습니까? (y/N): ").strip().lower()
    if change_settings == 'y':
        new_domain = input(f"새로운 도메인을 입력하세요 (현재: {checker.expected_domain}): ").strip()
        if new_domain:
            checker.expected_domain = new_domain
            print(f"🌐 도메인이 '{new_domain}'로 변경되었습니다.")
        
        new_author = input(f"새로운 작성자를 입력하세요 (현재: {checker.expected_author}): ").strip()
        if new_author:
            checker.expected_author = new_author
            print(f"👤 작성자가 '{new_author}'로 변경되었습니다.")
    
    print()
    
    # 검사 실행
    results = checker.check_directory(directory_path)
    
    if not results:
        print("❌ HTML 파일을 찾을 수 없습니다.")
        return
    
    # 자동 수정 옵션
    problem_files = [r for r in results if r.get('issues')]
    if problem_files:
        auto_fix = input(f"\n{len(problem_files)}개 파일에서 문제가 발견되었습니다. 자동으로 수정하시겠습니까? (y/N): ").strip().lower()
        if auto_fix == 'y':
            checker.auto_fix_problems()
            # 수정 후 재검사
            print("\n🔄 수정 후 재검사를 진행합니다...")
            checker.results = []
            checker.check_directory(directory_path)
    
    # 리포트 생성 옵션
    show_all = input("\n모든 파일을 리포트에 포함하시겠습니까? (y/N - 기본값: 문제가 있는 파일만): ").strip().lower()
    problems_only = show_all != 'y'
    
    # 리포트 생성
    print("\n📄 리포트를 생성합니다...")
    checker.generate_report(problems_only=problems_only)
    checker.generate_csv_report(problems_only=problems_only)
    
    # 간단한 요약 출력
    total_files = len(results)
    problem_files = [r for r in checker.results if r.get('issues')]
    fixed_files = [r for r in checker.results if r.get('fixed_issues')]
    
    print(f"\n✅ 검사 완료!")
    print(f"   - 총 {total_files}개 파일 검사")
    print(f"   - {len(problem_files)}개 파일에서 문제 발견")
    if fixed_files:
        print(f"   - {len(fixed_files)}개 파일 자동 수정 완료")
    
    if problem_files:
        print(f"\n⚠️ 남은 문제가 있는 파일들:")
        for result in problem_files[:5]:  # 처음 5개만 표시
            print(f"   - {result['file_path']}")
        if len(problem_files) > 5:
            print(f"   ... 외 {len(problem_files) - 5}개 파일")

if __name__ == "__main__":
    main()


검사할 디렉토리 경로를 입력하세요 (현재 디렉토리는 '.'): C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp
📂 'C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp' 디렉토리를 검사합니다...
🌐 예상 도메인: https://emfls.github.io/kor/report/travel/
👤 예상 작성자: Itagi
설정을 변경하시겠습니까? (y/N): n

총 10개의 HTML 파일을 발견했습니다.
검사 중... (1/10) C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp\benin-cotonou.html
검사 중... (2/10) C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp\mongolia-baganuur.html
검사 중... (3/10) C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp\mongolia-kharkhorin.html
검사 중... (4/10) C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp\mongolia-nalaikh.html
검사 중... (5/10) C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp\mongolia-tol.html
검사 중... (6/10) C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp\mongolia-tsagaan-suvarga.html
검사 중... (7/10) C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\temp\mongolia-ulaangom.ht

In [32]:
import os
import json
from bs4 import BeautifulSoup

def extract_info_from_html(file_path):
    """단일 HTML 파일에서 canonical 링크, 타이틀, 설명을 추출하는 함수"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            soup = BeautifulSoup(file, 'html.parser')
            
            # Canonical 링크 추출
            canonical_link = None
            canonical_tag = soup.find('link', rel='canonical')
            if canonical_tag and canonical_tag.has_attr('href'):
                canonical_link = canonical_tag['href']
            
            # 타이틀 추출
            title = None
            if soup.title:
                title = soup.title.string.strip()
            
            # 설명 추출 (meta description 우선, 없으면 첫 번째 p 태그)
            description = None
            meta_desc = soup.find('meta', attrs={'name': 'description'})
            if meta_desc and meta_desc.has_attr('content'):
                description = meta_desc['content'].strip()
            else:
                # 대안: 첫 번째 paragraph 텍스트
                p_tag = soup.find('p')
                if p_tag:
                    description = p_tag.get_text(strip=True)
            
            return {
                'link': canonical_link if canonical_link else '',
                'title': title if title else '',
                'description': description if description else ''
            }
    
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {file_path} - {str(e)}")
        return {
            'link': '',
            'title': '',
            'description': ''
        }

def extract_websites_from_folder(folder_path, output_json_path):
    """폴더 내 모든 HTML 파일을 처리하여 JSON 파일로 저장하는 함수"""
    websites = []
    
    # 폴더 존재 여부 확인
    if not os.path.exists(folder_path):
        print(f"폴더를 찾을 수 없습니다: {folder_path}")
        return
    
    # 폴더 내 HTML 파일들 처리
    html_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.html')]
    
    if not html_files:
        print(f"폴더에 HTML 파일이 없습니다: {folder_path}")
        return
    
    print(f"{len(html_files)}개의 HTML 파일을 처리 중...")
    
    for filename in html_files:
        file_path = os.path.join(folder_path, filename)
        print(f"처리 중: {filename}")
        info = extract_info_from_html(file_path)
        if info['title'] or info['link']:  # 최소한 타이틀이나 링크가 있는 경우만 추가
            websites.append(info)
    
    # JSON 파일로 저장
    try:
        with open(output_json_path, 'w', encoding='utf-8') as json_file:
            json.dump({'websites': websites}, json_file, ensure_ascii=False, indent=2)
        print(f"성공적으로 저장되었습니다: {output_json_path}")
        print(f"총 {len(websites)}개의 웹사이트 정보가 추출되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {str(e)}")

def main():
    """메인 실행 함수"""
    # 사용자 입력 받기
    folder_path = input("HTML 파일들이 있는 폴더 경로를 입력하세요: ").strip()
    output_path = input("출력할 JSON 파일명을 입력하세요 (예: websites.json): ").strip()
    
    if not output_path.endswith('.json'):
        output_path += '.json'
    
    # 웹사이트 정보 추출 및 저장
    extract_websites_from_folder(folder_path, output_path)

if __name__ == "__main__":
    # 직접 실행 예시
    # extract_websites_from_folder('html_folder_path', 'travel-websites.json')
    
    # 대화형 실행
    main()


HTML 파일들이 있는 폴더 경로를 입력하세요: C:\Users\USER\Documents\GitHub\emfls.github.io\kor\report\travel
출력할 JSON 파일명을 입력하세요 (예: websites.json): 1.json
5269개의 HTML 파일을 처리 중...
처리 중: albania-berat.html
처리 중: albania-durres.html
처리 중: albania-elbasan.html
처리 중: albania-fier.html
처리 중: albania-gjirokaster.html
처리 중: albania-kavaje.html
처리 중: albania-korce.html
처리 중: albania-kruje.html
처리 중: albania-kucove.html
처리 중: albania-kukes.html
처리 중: albania-lac.html
처리 중: albania-lezhe.html
처리 중: albania-lushnje.html
처리 중: albania-patos.html
처리 중: albania-peshkopi.html
처리 중: albania-pogradec.html
처리 중: albania-saranda.html
처리 중: albania-shkoder.html
처리 중: albania-tirana.html
처리 중: albania-vlore.html
처리 중: algeria-adrar.html
처리 중: algeria-ain-temouchent.html
처리 중: algeria-aindefla.html
처리 중: algeria-algiers.html
처리 중: algeria-annaba.html
처리 중: algeria-batna.html
처리 중: algeria-bechar.html
처리 중: algeria-bejaia.html
처리 중: algeria-biskra.html
처리 중: algeria-blida.html
처리 중: algeria-bordj-bou-arreridj.html
처리 중: alge

처리 중: azerbaijan-qakh.html
처리 중: azerbaijan-qazanbend.html
처리 중: azerbaijan-qirdalan.html
처리 중: azerbaijan-quba.html
처리 중: azerbaijan-qusar.html
처리 중: azerbaijan-sabirabad.html
처리 중: azerbaijan-satli.html
처리 중: azerbaijan-shabran.html
처리 중: azerbaijan-shabuz.html
처리 중: azerbaijan-shamakhi.html
처리 중: azerbaijan-shamkir.html
처리 중: azerbaijan-sharur.html
처리 중: azerbaijan-sheki.html
처리 중: azerbaijan-shirvan.html
처리 중: azerbaijan-shusha.html
처리 중: azerbaijan-siyazan.html
처리 중: azerbaijan-sumgait.html
처리 중: azerbaijan-tovus.html
처리 중: azerbaijan-udjar.html
처리 중: azerbaijan-yardimli.html
처리 중: azerbaijan-yevlax.html
처리 중: azerbaijan-zangilan.html
처리 중: azerbaijan-zaqatala.html
처리 중: azerbaijan-zardab.html
처리 중: bahamas-alice-town.html
처리 중: bahamas-andros.html
처리 중: bahamas-arthurs-town.html
처리 중: bahamas-cockburn-town.html
처리 중: bahamas-cooperstown.html
처리 중: bahamas-dunmore-town.html
처리 중: bahamas-freeport.html
처리 중: bahamas-governors-harbour.html
처리 중: bahamas-hailock.html
처리 중: bahamas-ma

처리 중: benin-toffo.html
처리 중: benin-touri.html
처리 중: benin-wawa.html
처리 중: bhutan-bumthang.html
처리 중: bhutan-chirang.html
처리 중: bhutan-damchu.html
처리 중: bhutan-gasa.html
처리 중: bhutan-gedu.html
처리 중: bhutan-gelephu.html
처리 중: bhutan-ha.html
처리 중: bhutan-jakar.html
처리 중: bhutan-lhuntse.html
처리 중: bhutan-lungtse.html
처리 중: bhutan-mongar.html
처리 중: bhutan-paro.html
처리 중: bhutan-phuentsholing.html
처리 중: bhutan-punakha.html
처리 중: bhutan-samdrup-jongkhar.html
처리 중: bhutan-samtse.html
처리 중: bhutan-sarpang.html
처리 중: bhutan-tashiyangtse.html
처리 중: bhutan-thendhu.html
처리 중: bhutan-thimphu.html
처리 중: bhutan-trashigang.html
처리 중: bhutan-trongsa.html
처리 중: bhutan-wangdue.html
처리 중: bolivia-apolo.html
처리 중: bolivia-beni.html
처리 중: bolivia-cobija.html
처리 중: bolivia-cochabamba.html
처리 중: bolivia-copacabana.html
처리 중: bolivia-guayaramerin.html
처리 중: bolivia-lapaz.html
처리 중: bolivia-montero.html
처리 중: bolivia-oruro.html
처리 중: bolivia-potosi.html
처리 중: bolivia-riberalta.html
처리 중: bolivia-rurrenabaque.htm

처리 중: cape-verde-espargos.html
처리 중: cape-verde-joao-teves.html
처리 중: cape-verde-mindelo.html
처리 중: cape-verde-mosteiros.html
처리 중: cape-verde-nova-sintra.html
처리 중: cape-verde-pombas.html
처리 중: cape-verde-porto-novo.html
처리 중: cape-verde-praia.html
처리 중: cape-verde-ribeira-brava.html
처리 중: cape-verde-ribeira-grande.html
처리 중: cape-verde-sal-rei.html
처리 중: cape-verde-santa-maria.html
처리 중: cape-verde-sao-domingos.html
처리 중: cape-verde-sao-filipe.html
처리 중: cape-verde-tarrafal.html
처리 중: car-bambari.html
처리 중: car-bangui.html
처리 중: car-berberati.html
처리 중: car-bimbo.html
처리 중: car-birao.html
처리 중: car-bossangoa.html
처리 중: car-bouar.html
처리 중: car-bria.html
처리 중: car-kaga-bandoro.html
처리 중: car-mbaiki.html
처리 중: car-mobaye.html
처리 중: car-ndele.html
처리 중: car-nola.html
처리 중: car-obo.html
처리 중: car-sibut.html
처리 중: chad-abeche.html
처리 중: chad-baibokoum.html
처리 중: chad-beida.html
처리 중: chad-bongor.html
처리 중: chad-doba.html
처리 중: chad-faya.html
처리 중: chad-goz-beida.html
처리 중: chad-korotom.ht

처리 중: czech-trebic.html
처리 중: czech-trutnov.html
처리 중: czech-usti-nad-labem.html
처리 중: czech-zlin.html
처리 중: denmark-aalborg.html
처리 중: denmark-aarhus.html
처리 중: denmark-albertslund.html
처리 중: denmark-allerod.html
처리 중: denmark-bornholm.html
처리 중: denmark-brondby.html
처리 중: denmark-copenhagen.html
처리 중: denmark-esbjerg.html
처리 중: denmark-fredensborg.html
처리 중: denmark-frederiksberg.html
처리 중: denmark-frederikssund.html
처리 중: denmark-fureso.html
처리 중: denmark-gladsaxe.html
처리 중: denmark-glostrup.html
처리 중: denmark-halsnaes.html
처리 중: denmark-herning.html
처리 중: denmark-hillerod.html
처리 중: denmark-hilleroed.html
처리 중: denmark-holbaek.html
처리 중: denmark-horsholm.html
처리 중: denmark-hvidovre.html
처리 중: denmark-ishoj.html
처리 중: denmark-kolding.html
처리 중: denmark-loedovre.html
처리 중: denmark-lyngby.html
처리 중: denmark-naestved.html
처리 중: denmark-odense.html
처리 중: denmark-randers.html
처리 중: denmark-roskilde.html
처리 중: denmark-rudersdal.html
처리 중: denmark-silkeborg.html
처리 중: denmark-skagen.html
처

...

처리 중: russia-astrakhan.html
처리 중: russia-barnaul.html
처리 중: russia-batumi.html
처리 중: russia-belgorod.html
처리 중: russia-bryansk.html
처리 중: russia-cheboksary.html
처리 중: russia-chelyabinsk.html
처리 중: russia-cherepovets.html
처리 중: russia-chita.html
처리 중: russia-irkutsk.html
처리 중: russia-ivanovo.html
처리 중: russia-izhevsk.html
처리 중: russia-kaliningrad.html
처리 중: russia-kaluga.html
처리 중: russia-kazan.html
처리 중: russia-kemerovo.html
처리 중: russia-khabarovsk.html
처리 중: russia-kirov.html
처리 중: russia-kolpino.html
처리 중: russia-krasnodar.html
처리 중: russia-krasnoyarsk.html
처리 중: russia-kursk.html
처리 중: russia-lipetsk.html
처리 중: russia-magnitogorsk.html
처리 중: russia-makhachkala.html
처리 중: russia-moscow.html
처리 중: russia-murmansk.html
처리 중: russia-naberezhnyechelny.html
처리 중: russia-nizhny-novgorod.html
처리 중: russia-nizhny-tagil.html
처리 중: russia-novokuznetsk.html
처리 중: russia-novosibirsk.html
처리 중: russia-omsk.html
처리 중: russia-orenburg.html
처리 중: russia-oryol.html
처리 중: russia-penza.html
처리 중: russi

처리 중: somalia-lasanod.html
처리 중: somalia-luuq.html
처리 중: somalia-merca.html
처리 중: somalia-mogadishu.html
처리 중: south-africa-albertyn.html
처리 중: south-africa-bellville.html
처리 중: south-africa-benoni.html
처리 중: south-africa-bethulie.html
처리 중: south-africa-bloemfontein.html
처리 중: south-africa-boksburg.html
처리 중: south-africa-breetz.html
처리 중: south-africa-cape-town.html
처리 중: south-africa-carletonville.html
처리 중: south-africa-centurion.html
처리 중: south-africa-durban.html
처리 중: south-africa-east-london.html
처리 중: south-africa-ermelo.html
처리 중: south-africa-george.html
처리 중: south-africa-germiston.html
처리 중: south-africa-heidelberg.html
처리 중: south-africa-hermanus.html
처리 중: south-africa-johannesburg.html
처리 중: south-africa-kimberley.html
처리 중: south-africa-klerksdorp.html
처리 중: south-africa-knysna.html
처리 중: south-africa-krugersdorp.html
처리 중: south-africa-lichtenburg.html
처리 중: south-africa-louistriehardt.html
처리 중: south-africa-middelburg.html
처리 중: south-africa-mitchells-plain.html
처리 

처리 중: switzerland-schaffhausen.html
처리 중: switzerland-schwyz.html
처리 중: switzerland-solothurn.html
처리 중: switzerland-steinamrhein.html
처리 중: switzerland-stgallen.html
처리 중: switzerland-thun.html
처리 중: switzerland-uster.html
처리 중: switzerland-vevey.html
처리 중: switzerland-winterthur.html
처리 중: switzerland-zug.html
처리 중: switzerland-zurich.html
처리 중: syria-afrin.html
처리 중: syria-al-bab.html
처리 중: syria-aleppo.html
처리 중: syria-ariha.html
처리 중: syria-azaz.html
처리 중: syria-dabiq.html
처리 중: syria-damascus.html
처리 중: syria-daraa.html
처리 중: syria-deir-ez-zor.html
처리 중: syria-hama.html
처리 중: syria-homs.html
처리 중: syria-idlib.html
처리 중: syria-jarabulus.html
처리 중: syria-jisr-al-shughur.html
처리 중: syria-kanshaykhun.html
처리 중: syria-kobani.html
처리 중: syria-latakia.html
처리 중: syria-maaloula.html
처리 중: syria-maarat-misrin.html
처리 중: syria-maarat-numan.html
처리 중: syria-maarrat-misrin.html
처리 중: syria-manbij.html
처리 중: syria-masyaf.html
처리 중: syria-mayadeen.html
처리 중: syria-raqqa.html
처리 중: syria-saraqi

처리 중: tunisia-siliana.html
처리 중: tunisia-skhira.html
처리 중: tunisia-sousse.html
처리 중: tunisia-tabarka.html
처리 중: tunisia-takrouna.html
처리 중: tunisia-tataouine.html
처리 중: tunisia-testour.html
처리 중: tunisia-tozeur.html
처리 중: tunisia-tunis.html
처리 중: tunisia-yasmine-hammamet.html
처리 중: tunisia-zaghouan.html
처리 중: tunisia-zarzis.html
처리 중: turkey-adana.html
처리 중: turkey-afyonkarahisar.html
처리 중: turkey-agri.html
처리 중: turkey-amasya.html
처리 중: turkey-ankara.html
처리 중: turkey-antalya.html
처리 중: turkey-ardahan.html
처리 중: turkey-artvin.html
처리 중: turkey-aydin.html
처리 중: turkey-balikesir.html
처리 중: turkey-batman.html
처리 중: turkey-bayburt.html
처리 중: turkey-bingol.html
처리 중: turkey-bitlis.html
처리 중: turkey-bursa.html
처리 중: turkey-denizli.html
처리 중: turkey-diyarbakir.html
처리 중: turkey-edirne.html
처리 중: turkey-elazig.html
처리 중: turkey-erzurum.html
처리 중: turkey-eskisehir.html
처리 중: turkey-gaziantep.html
처리 중: turkey-gulluk.html
처리 중: turkey-ios.html
처리 중: turkey-istanbul.html
처리 중: turkey-izmir.html


처리 중: uzbekistan-bukhara.html
처리 중: uzbekistan-chirchik.html
처리 중: uzbekistan-denau.html
처리 중: uzbekistan-fergana.html
처리 중: uzbekistan-jizzakh.html
처리 중: uzbekistan-karshi.html
처리 중: uzbekistan-kashan.html
처리 중: uzbekistan-kokand.html
처리 중: uzbekistan-margilan.html
처리 중: uzbekistan-namangan.html
처리 중: uzbekistan-navoi.html
처리 중: uzbekistan-nukus.html
처리 중: uzbekistan-quba.html
처리 중: uzbekistan-samarkand.html
처리 중: uzbekistan-shakhrisabz.html
처리 중: uzbekistan-tashkent.html
처리 중: uzbekistan-termez.html
처리 중: uzbekistan-urgench.html
처리 중: vanuatu-isangel.html
처리 중: vanuatu-lakatoro.html
처리 중: vanuatu-longana.html
처리 중: vanuatu-luganville.html
처리 중: vanuatu-portvila.html
처리 중: vanuatu-sola.html
처리 중: vatican-vaticancity.html
처리 중: venezuela-acarigua.html
처리 중: venezuela-aragua.html
처리 중: venezuela-barcelona.html
처리 중: venezuela-barinas.html
처리 중: venezuela-barquisimeto.html
처리 중: venezuela-cabimas.html
처리 중: venezuela-caboredero.html
처리 중: venezuela-caracas.html
처리 중: venezuela-carupano.h